# **Reto II**

### 1. Dataset

Los datos de origen son proporcionados en un archivos csv:

* udfs: dataset con datos de operaciones financieras.

### 2. Columnas y significado:

* nb: número de referencia de la operación.
* contract: identificador de contrato.
* udf_ref: identificador de operación de trading.
* fmly: familia a la que pertenece la operación financiera.
* grp: grupo al que pertenece la operación financiera.
* type: tipo de operación financiera.
* country: país de origen de la operación.
* udf_name: campo informado en el registro.
* num_value: valor numérico.
* string_value: valor de cadena de caracteres.
* date_value: valor de fecha.
* data_timestamp_part: marca temporal.
* data_date_part: fecha en la que se almacena la información.
* source_system: fuente de los datos.

### 3. Descripción del problema:

Si hacemos una visión general a nuestro conjunto de datos, podemos observar como hay hasta 10 registros (filas) para cada valor de *nb*, donde cada registro solo da información para un valor de *udf_name*. Esto es un gasto innecesario de almacenamiento y computación, además de complicar los futuros cálculos derivados de estos datos. Por esta razón, necesitamos convertir estos registros con el mismo *nb* a un solo registro.

Nuestro dataframe final tendrá que contener las siguientes columnas: `nb, M_CCY, M_CLIENT, M_CRDTCHRG, M_DIRECTIAV, M_DISCMARGIN, M_LIQDTYCHRG, M_MVA, M_RVA, M_SELLER, M_SUCURSAL`

* nb: debe contener el número de referencia de la operación.
* M_CLIENT, M_SELLER, M_CCY, M_SUCURSAL: deben mapear el valor de *string_value*
* M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA: deben mapear el valor de *num_value*


Una vez tengamos este resultado, necesitaremos eliminar las operaciones que no tengan informados ninguno de los siguientes campos:

M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

No informados en este caso significa que o son valores nulos, vacíos o 0, en el caso de los campos numéricos.

### 4. Reto:

* Obtener un dataframe final que contenga las columnas indicadas, con un registro por *nb* y con los valores correctos mapeados.
* Las operaciones con los campos M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, , M_MVA, M_RVA, M_SELLER no informados no deben existir.
* Hacerlo de la manera más eficiente posible a nivel computacional.

**NOTA:** Cada uno de los pasos descritos en el problema pueden efectuarse en una sola línea.

### Inicialización de SparkSession:

In [1]:
import org.apache.spark.sql.SparkSession

val spark = SparkSession.builder()
                        .appName("Reto 2")
                        .master("local")
                        .getOrCreate()

Intitializing Scala interpreter ...

Spark Web UI available at http://L2110017.bosonit.local:4041
SparkContext available as 'sc' (version = 3.1.2, master = local[*], app id = local-1641542646206)
SparkSession available as 'spark'


import org.apache.spark.sql.SparkSession
spark: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@caee0c3


In [2]:
val udfs = spark.read.format("csv")
                     .option("header", "true")
                     .option("delimiter", ";")
                     .load("./reto2/udfs.csv")
udfs.where(col("udf_name")==="M_DISCMARGIN").show(100)

+---------+---------+---------+----+-----+-----+-------+------------+---------------+------------+----------+-------------------+--------------+-------------+
|       nb| contract|  udf_ref|fmly|  grp| type|country|    udf_name|      num_value|string_value|date_value|data_timestamp_part|data_date_part|source_system|
+---------+---------+---------+----+-----+-----+-------+------------+---------------+------------+----------+-------------------+--------------+-------------+
|  2222222|  2222222|  2222222| IRD|  IRS| null|    ESP|M_DISCMARGIN|             10|        NULL|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
|  3815982|  3672136|  4951879| SCF|  SCF|  SCF|    ESP|M_DISCMARGIN| 0.000000000000|        NULL|      NULL|     20200406103021|    2020-12-30|        Mx3EU|
| 14773283| 13940488| 21807924| IRD|LN_BR| null|    ESP|M_DISCMARGIN|            200|        NULL|      NULL|     20200406103021|    2020-12-30|        Mx3EU|
| 18343978| 17356077| 27518638| IRD|  IRS| nul

udfs: org.apache.spark.sql.DataFrame = [nb: string, contract: string ... 12 more fields]


+---------+-----+--------+---------------+-----------------+--------------+---------------+--------------+--------------+----------+----------+
|       nb|M_CCY|M_CLIENT|     M_CRDTCHRG|      M_DIRECTIAV|  M_DISCMARGIN|   M_LIQDTYCHRG|         M_MVA|         M_RVA|  M_SELLER|M_SUCURSAL|
+---------+-----+--------+---------------+-----------------+--------------+---------------+--------------+--------------+----------+----------+
| 20513130| null|    CCMO| 0.000000000000|   0.000000000000|            10| 0.000000000000|            20|0.000000000000|      WATT|      5493|
| 18710605|  MXN|    null|           null|   0.000000000000|          null|           null|          null|          null|      AMAM|      null|
| 20533916| null|    CCMO| 0.000000000000|   0.000000000000|            50| 0.000000000000|            30|0.000000000000|      WATT|      1999|
| 23097010|  EUR|    null|           null|   0.000000000000|          null|           null|          null|          null|      AMAM|    

resultado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]


In [98]:
val a= udfs.groupBy("nb").pivot("udf_name").agg(Map("num_value"->"first"))
        .union(udfs.groupBy("nb").pivot("udf_name").agg(Map("string_value"->"first")))
        .groupBy("nb").agg(last("M_CCY").alias("M_CCY"),last("M_CLIENT").alias("M_CLIENT"),first("M_CRDTCHRG").alias("M_CRDTCHRG"),first("M_DIRECTIAV").alias("M_DIRECTIAV"),first("M_DISCMARGIN").alias("M_DISCMARGIN"),first("M_LIQDTYCHRG").alias("M_LIQDTYCHRG"),first("M_MVA").alias("M_MVA"),first("M_RVA").alias("M_RVA"),last("M_SELLER").alias("M_SELLER"),last("M_SUCURSAL").alias("M_SUCURSAL"))
        .drop("M_PRUEBA")
        .filter(col("M_DISCMARGIN").isNotNull || col("M_DIRECTIAV").isNotNull || col("M_LIQDTYCHRG").isNotNull || col("M_CRDTCHRG").isNotNull || col("M_MVA").isNotNull || col("M_RVA").isNotNull || col("M_SELLER").isNotNull || round(col("M_DISCMARGIN"),0)=!=0 || round(col("M_DIRECTIAV"),0)=!=0 || round(col("M_LIQDTYCHRG"),0)=!=0 || round(col("M_CRDTCHRG"),0)=!=0 || round(col("M_MVA"),0)=!=0 || round(col("M_RVA"),0)=!=0)
a.count()
//M_DISCMARGIN, M_DIRECTIAV, M_LIQDTYCHRG, M_CRDTCHRG, M_MVA, M_RVA, M_SELLER

a: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]
res85: Long = 61


In [99]:
val a= udfs.groupBy("nb").pivot("udf_name").agg(Map("num_value"->"first"))
        .union(udfs.groupBy("nb").pivot("udf_name").agg(Map("string_value"->"first")))
        .groupBy("nb").agg(last("M_CCY").alias("M_CCY"),last("M_CLIENT").alias("M_CLIENT"),first("M_CRDTCHRG").alias("M_CRDTCHRG"),first("M_DIRECTIAV").alias("M_DIRECTIAV"),first("M_DISCMARGIN").alias("M_DISCMARGIN"),first("M_LIQDTYCHRG").alias("M_LIQDTYCHRG"),first("M_MVA").alias("M_MVA"),first("M_RVA").alias("M_RVA"),last("M_SELLER").alias("M_SELLER"),last("M_SUCURSAL").alias("M_SUCURSAL"))
        .drop("M_PRUEBA")
        .filter(col("M_SELLER").isNotNull || round(col("M_DISCMARGIN"),0)=!=0 || round(col("M_DIRECTIAV"),0)=!=0 || round(col("M_LIQDTYCHRG"),0)=!=0 || round(col("M_CRDTCHRG"),0)=!=0 || round(col("M_MVA"),0)=!=0 || round(col("M_RVA"),0)=!=0)
a.count()

a: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]
res86: Long = 60


In [59]:
import pyspark.sql.types.StringType
val a= udfs.groupBy("nb").pivot("udf_name").agg(collect_list("string_value").cast(StringType()))
.drop("M_PRUEBA")
a.show(50)

<console>: 29: error: not found: value pyspark

In [76]:
val a= udfs.groupBy("nb").pivot("udf_name").agg().drop("M_PRUEBA")
                    
a.show(50)

<console>: 2: error: ')' expected but string literal found.

In [73]:
udfs.show()

+-------+--------+--------+----+-----+----+-------+------------+---------------+------------+----------+-------------------+--------------+-------------+
|     nb|contract| udf_ref|fmly|  grp|type|country|    udf_name|      num_value|string_value|date_value|data_timestamp_part|data_date_part|source_system|
+-------+--------+--------+----+-----+----+-------+------------+---------------+------------+----------+-------------------+--------------+-------------+
| 444444|    3333|28786653| IRD|LN_BR|null|    ESP|       M_CCY|           NULL|        null|      NULL|     20201128041303|    2020-12-30|        Mx3EU|
|2222222| 2222222| 2222222| IRD|  IRS|null|    ESP|  M_CRDTCHRG|30.000000000000|        NULL|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
|2222222| 2222222| 2222222| IRD|  IRS|null|    ESP|    M_SELLER|           NULL|  LB_TLECLER|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
|2222222| 2222222| 2222222| IRD|  IRS|null|    ESP|M_LIQDTYCHRG|50.000000000

In [50]:
a.where($"nb" === 23037162).show()

+--------+-----+--------+----------+-----------+------------+------------+-----+-----+--------+----------+
|      nb|M_CCY|M_CLIENT|M_CRDTCHRG|M_DIRECTIAV|M_DISCMARGIN|M_LIQDTYCHRG|M_MVA|M_RVA|M_SELLER|M_SUCURSAL|
+--------+-----+--------+----------+-----------+------------+------------+-----+-----+--------+----------+
|23037162|[EUR]|      []|        []|         []|          []|          []|   []|   []|  [AMAM]|        []|
+--------+-----+--------+----------+-----------+------------+------------+-----+-----+--------+----------+



In [53]:
udfs.show(100)

+--------+----------+--------+----+-----+-----+-------+------------+-----------------+------------+----------+-------------------+--------------+-------------+
|      nb|  contract| udf_ref|fmly|  grp| type|country|    udf_name|        num_value|string_value|date_value|data_timestamp_part|data_date_part|source_system|
+--------+----------+--------+----+-----+-----+-------+------------+-----------------+------------+----------+-------------------+--------------+-------------+
|  444444|      3333|28786653| IRD|LN_BR| null|    ESP|       M_CCY|             NULL|        null|      NULL|     20201128041303|    2020-12-30|        Mx3EU|
| 2222222|   2222222| 2222222| IRD|  IRS| null|    ESP|  M_CRDTCHRG|  30.000000000000|        NULL|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
| 2222222|   2222222| 2222222| IRD|  IRS| null|    ESP|    M_SELLER|             NULL|  LB_TLECLER|      NULL|     20210203032054|    2020-12-30|        Mx3EU|
| 2222222|   2222222| 2222222| IRD|  IRS

|19772400|  18724280|29554565|CURR|  FXD|SWLEG|    ESP|  M_SUCURSAL|             NULL|        1999|      NULL|     20201110033853|    2020-12-30|        Mx3EU|
+--------+----------+--------+----+-----+-----+-------+------------+-----------------+------------+----------+-------------------+--------------+-------------+
only showing top 100 rows



### Resultado:

**INSTRUCCIONES**: El DataFrame resultante debe almacenarse en la variable `resultado`, sustituyendo el valor `None` por el código que consideréis oportuno. De esta forma podréis comprobar si el resultado es correcto.

In [57]:
//bien
val resultado = udfs.groupBy("nb").pivot("udf_name").agg(Map("num_value"->"first"))
        .union(udfs.groupBy("nb").pivot("udf_name").agg(Map("string_value"->"first")))
        .groupBy("nb").agg(last("M_CCY").alias("M_CCY"),last("M_CLIENT").alias("M_CLIENT"),first("M_CRDTCHRG").alias("M_CRDTCHRG"),first("M_DIRECTIAV").alias("M_DIRECTIAV"),first("M_DISCMARGIN").alias("M_DISCMARGIN"),first("M_LIQDTYCHRG").alias("M_LIQDTYCHRG"),first("M_MVA").alias("M_MVA"),first("M_RVA").alias("M_RVA"),last("M_SELLER").alias("M_SELLER"),last("M_SUCURSAL").alias("M_SUCURSAL"))
        .drop("M_PRUEBA")
        .filter(col("M_SELLER")=!="NULL" || col("M_SELLER")=!="NULL" || col("M_SELLER").isNotNull || col("M_DISCMARGIN")=!=0 || col("M_DIRECTIAV")=!=0 || col("M_LIQDTYCHRG")=!=0 || col("M_CRDTCHRG")=!=0 || col("M_MVA")=!=0 || col("M_RVA")=!=0)


//resultado.orderBy("nb").show()

resultado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]


In [54]:
//bien2 
val resultado= udfs.withColumn("num_value",when(col("num_value")==="NULL",null).otherwise(col("num_value")))
           .withColumn("aux",when(col("num_value").isNull,col("string_value")).otherwise(col("num_value")))
           .groupBy("nb").pivot("udf_name").agg(Map("aux"->"first"))
           .drop("M_PRUEBA")
           .filter(col("M_SELLER")=!="" || col("M_SELLER")=!="NULL" || col("M_SELLER").isNotNull || col("M_DISCMARGIN")=!=0 || col("M_DIRECTIAV")=!=0 || col("M_LIQDTYCHRG")=!=0 || col("M_CRDTCHRG")=!=0 || col("M_MVA")=!=0 || col("M_RVA")=!=0)

//resultado.orderBy("nb").show()

+---------+-----+--------+---------------+-----------------+---------------+---------------+--------------+--------------+----------+----------+
|       nb|M_CCY|M_CLIENT|     M_CRDTCHRG|      M_DIRECTIAV|   M_DISCMARGIN|   M_LIQDTYCHRG|         M_MVA|         M_RVA|  M_SELLER|M_SUCURSAL|
+---------+-----+--------+---------------+-----------------+---------------+---------------+--------------+--------------+----------+----------+
| 10000001| null|    NULL|             20|   0.000000000000|             10|             30|0.000000000000|0.000000000000|   SELLER1|      1999|
| 10000009| null|    NULL|             20|   0.000000000000|             10|             30|0.000000000000|0.000000000000|   SELLER9|      1999|
|111222333| null|    MMMM| 0.000000000000|             9056| 0.000000000000| 0.000000000000|            20|          8956|    VVVVVV|      1212|
| 14773283| null|    NULL|             10|               10|            200|             10|            10|             5|      nu

resultado: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [nb: string, M_CCY: string ... 9 more fields]


In [55]:
assert(resultado.count() == 60)
assert(resultado.columns.size == 11)
assert(resultado.columns(4) == "M_DIRECTIAV")
assert(resultado.select("M_SELLER").filter($"nb" === 23037162).first.getString(0) == "AMAM")
assert(resultado.select("M_SELLER").filter($"nb" === 19665186).first.getString(0) == "LB_VSTAVRE")
assert(resultado.select("M_RVA").filter($"nb" === 444111222).first.getString(0) == "8956")